In [ ]:
import json
import requests

In [ ]:
with open("./interface/ai_soluition_certification.json", 'r') as outfile:
    interface = json.load(outfile)

In [ ]:
params = {"name":interface['name'],
"ras_version_id":interface['version_id'],
"workspace_name":interface['workspace_name']
}
params

In [ ]:
# a = {"workspace_name" :interface['workspace_name'], "page_size": 10}
# cass_list = requests.get(interface['server_uri'] + f"api/v1/cass", params=a) 
# cass_list.json()

In [ ]:
# for list_name in cass_list.json()['cass']:
#     print(list_name['name'])
#     print(list_name['id'])

In [ ]:
# requests.delete(interface['server_uri'] + f"/api/v1/cass/{interface['version_id']}")

In [ ]:
# cass 등록
cass = requests.post(interface['server_uri'] + 'api/v1/cass', params=params)
cass.json()

In [ ]:
solution_instance_result = {
    "name":cass.json()['name'],
    "cas_id":cass.json()['id']
}

In [ ]:
#stream result로 바꾸기
stream = requests.post(interface['server_uri'] +'api/v1/stream', params=solution_instance_result)
stream.json()

In [ ]:
import yaml

# YAML 파일 경로
yaml_file_path = './solution_metadata.yaml'

# YAML 파일을 읽어서 파싱
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

In [ ]:
# 솔메를 읽어서 metadata_json에 넣기
data = {
  "metadata_json": yaml_data
}
data =json.dumps(data)

In [ ]:
pipeline = requests.post(interface['server_uri'] + f"api/v1/stream/{stream.json()['id']}/start", data=data)
pipeline.json()

In [ ]:
info = requests.get(interface['server_uri'] + f"api/v1/stream/{pipeline.json()['id']}/info")
info.json()['status']

In [ ]:
pipeline.json()['train_dataset_uri'][0][0]

In [ ]:
def split_s3_path(s3_path):
    # 's3://'를 제거하고 '/'를 기준으로 첫 부분을 분리하여 bucket과 나머지 경로를 얻습니다.
    path_parts = s3_path.replace('s3://', '').split('/', 1)
    bucket = path_parts[0]
    rest_of_the_path = path_parts[1]
    return bucket, rest_of_the_path

In [ ]:
split_s3_path(pipeline.json()['train_artifact_uri'])

In [ ]:
split_s3_path(pipeline.json()['train_artifact_uri'])

In [ ]:
import boto3

s3 = boto3.client('s3')
s3_bucket = split_s3_path(pipeline.json()['train_artifact_uri'])[0]
s3_prefix = split_s3_path(pipeline.json()['train_artifact_uri'])[1]

# S3 버킷에서 파일 목록 가져오기
objects = s3.list_objects(Bucket=s3_bucket, Prefix=s3_prefix)

# 파일 다운로드
for obj in objects.get('Contents', []):
    key = obj['Key']
    filename = key.split('/')[-1]  # 파일 이름 추출
    s3.download_file(s3_bucket, key, filename)
    print(f'Downloaded: {filename}')

In [ ]:
!tar -xvf ./train_artifacts.tar.gz

In [ ]:
!cp .train_artifacts/ ../../ -r

In [ ]:
!python ../../main.py --mode 'inference'

In [ ]:
!pip list | grep requests

In [ ]:
!tar -cvf ./inference.tar ../../.inference_artifacts/

In [ ]:
split_s3_path(yaml_data['pipeline'][1]['artifact_uri'])

In [ ]:
bucket_name = 'your-bucket-name'  # S3 버킷 이름
file_name = 'local-file-path'  # 로컬 시스템의 파일 경로
object_name = 's3-object-name'  # S3에 저장될 객체 이름 (예: 'folder/subfolder/filename')